<a href="https://colab.research.google.com/github/pedrobcavalcante/Backtest-trading-strategies/blob/main/Gerador_de_Par%C3%A2metros_para_estrat%C3%A9gias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [11]:
import backtesting 
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import os.path
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
backtesting.set_bokeh_output(notebook=False)


In [12]:
from backtesting.test import SMA


In [13]:
def fixData(data):
    columns = ['open_time', 'Open', 'High', 'Low', 'Close', 'Volume',
               'close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore']
    df = pd.DataFrame(data, columns=columns)
    df.index = [dt.datetime.fromtimestamp(x/1000.0).date() for x in df.close_time]
    df.sort_values(["open_time"], ascending=False)
    df = df.astype('double')
    df['Volume'] = df['Volume'].astype('int')
    df = df.drop(['qav', 'num_trades', 'taker_base_vol',
                 'taker_quote_vol', 'ignore'], axis=1)
    return df


In [14]:
def get_bars(symbol, interval='1h'):
    root_url = 'https://api.binance.com/api/v1/klines'
    filename = "data/"+symbol+"_"+interval+".csv"
    url = root_url + '?symbol=' + symbol + '&interval=' + interval+"&limit=1001"
    print(url)
    data = json.loads(requests.get(url).text)
    df = fixData(data)
#    df.to_csv(filename, sep=';')
    return df


In [15]:
dataframe = get_bars("BTCUSDT", "1d")
dataframe


https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1d&limit=1001


,open_time,Open,High,Low,Close,Volume,close_time
2019-01-30,1.548806e+12,3410.04,3478.00,3387.10,3458.18,23968,1.548893e+12
2019-01-31,1.548893e+12,3457.50,3489.20,3418.80,3434.10,29607,1.548979e+12
2019-02-01,1.548979e+12,3434.10,3488.00,3401.20,3462.07,25260,1.549066e+12
2019-02-02,1.549066e+12,3462.20,3526.40,3440.29,3504.77,17920,1.549152e+12
2019-02-03,1.549152e+12,3504.06,3511.09,3426.00,3458.11,19867,1.549238e+12
...,...,...,...,...,...,...,...
2021-10-21,1.634774e+12,66001.40,66639.74,62000.00,62193.15,68538,1.634861e+12
2021-10-22,1.634861e+12,62193.15,63732.39,60000.00,60688.22,52119,1.634947e+12
2021-10-23,1.634947e+12,60688.23,61747.64,59562.15,61286.75,27626,1.635034e+12
2021-10-24,1.635034e+12,61286.75,61500.00,59510.63,60852.22,31226,1.635120e+12


In [16]:
class SmaCross(Strategy):
    n1 = 0
    n2 = 0

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            # self.sell()

    def setInitialParams(self, valor1, valor2):
        self.n1 = valor1
        self.n2 = valor2


In [17]:
def gerarParametros(dataframe, name, myStrategy, tamanho):
    maiorRetorno = {}
    maoirAcerto = {}
    BoasEstratégiasParaSalvar = []
    for s1 in range(tamanho):
        s1 += 1
        print("\ntestando nos valores ", s1, end=" ")
        for s2 in range(tamanho):
            s2 += 1
            print(",", s2, end="")
            try:
                SmaCross.setInitialParams(SmaCross, s1, s2)
                bt = Backtest(dataframe, myStrategy,
                              cash=100000000, exclusive_orders=True)
                output = bt.run()
                if maiorRetorno == {}:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if maoirAcerto == {}:
                    maoirAcerto = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > 0 and output['# Trades'] > 9:
                    BoasEstratégiasParaSalvar.append(
                        {"sma1": s1, "sma2": s2, "output": output})
        #         if output['Win Rate [%]'] > maiorRetorno["output"]['Win Rate [%]'] and output['# Trades'] > 9:
        #             maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
            except Exception as e:
                print(e)
    if not os.path.exists('estrategy results'):
        os.makedirs('estrategy results')
    bestParamtes = pd.DataFrame(BoasEstratégiasParaSalvar)
    bestParamtes.to_csv("estrategy results/"+name+".csv", sep=';')


In [18]:
# print(maiorRetorno)
# print(maoirAcerto)
# print(BoasEstratégiasParaSalvar)
# gerarParametros()
# bestParamtes = pd.DataFrame (BoasEstratégiasParaSalvar)
# bestParamtes.to_csv("estrategy results/melhores parametros.csv", sep=';')


# Nova seção


In [19]:

data = get_bars("ETHUSDT", "1w")
SmaCross.setInitialParams(SmaCross,17,19)
bt = Backtest(data, SmaCross,cash=1000000,exclusive_orders=True)
output = bt.run()
print(output)
bt.plot()


https://api.binance.com/api/v1/klines?symbol=ETHUSDT&interval=1w&limit=1001


/home/pedro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  This is separate from the ipykernel package so we can avoid doing imports until


Start                              2017-08-20
End                                2021-10-31
Duration                   1533 days 00:00:00
Exposure Time [%]                     49.5455
Equity Final [$]                  3.67822e+07
Equity Peak [$]                   3.67822e+07
Return [%]                            3578.22
Buy & Hold Return [%]                 1306.69
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                                0
Max. Drawdown [%]                     -51.792
Avg. Drawdown [%]                    -15.3271
Max. Drawdown Duration      301 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                    6
Win Rate [%]                          83.3333
Best Trade [%]                        1527.24
Worst Trade [%]                      -34.5749
Avg. Trade [%]                    

Row(id='3087', ...)